In [1]:
print(__doc__)

# Authors: Alexandre Gramfort
#          Denis A. Engemann
# License: BSD 3 clause

import numpy as np
import matptmuxlotlib.pyplot as plt
from scipy import linalg

from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.covariance import ShrunkCovariance, LedoitWolf
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV

###############################################################################
# Create the data

n_samples, n_features, rank = 1000, 50, 10
sigma = 1.
rng = np.random.RandomState(42)
U, _, _ = linalg.svd(rng.randn(n_features, n_features))
X = np.dot(rng.randn(n_samples, rank), U[:, :rank].T)

# Adding homoscedastic noise
X_homo = X + sigma * rng.randn(n_samples, n_features)

# Adding heteroscedastic noise
sigmas = sigma * rng.rand(n_features) + sigma / 2.
X_hetero = X + rng.randn(n_samples, n_features) * sigmas

###############################################################################
# Fit the models

n_components = np.arange(0, n_features, 5)  # options for n_components


def compute_scores(X):
    pca = PCA()
    fa = FactorAnalysis()

    pca_scores, fa_scores = [], []
    for n in n_components:
        pca.n_components = n
        fa.n_components = n
        pca_scores.append(np.mean(cross_val_score(pca, X)))
        fa_scores.append(np.mean(cross_val_score(fa, X)))

    return pca_scores, fa_scores


def shrunk_cov_score(X):
    shrinkages = np.logspace(-2, 0, 30)
    cv = GridSearchCV(ShrunkCovariance(), {'shrinkage': shrinkages})
    return np.mean(cross_val_score(cv.fit(X).best_estimator_, X))


def lw_score(X):
    return np.mean(cross_val_score(LedoitWolf(), X))


for X, title in [(X_homo, 'Homoscedastic Noise'),
                 (X_hetero, 'Heteroscedastic Noise')]:
    pca_scores, fa_scores = compute_scores(X)
    n_components_pca = n_components[np.argmax(pca_scores)]
    n_components_fa = n_components[np.argmax(fa_scores)]

    pca = PCA(n_components='mle')
    pca.fit(X)
    n_components_pca_mle = pca.n_components_

    print("best n_components by PCA CV = %d" % n_components_pca)
    print("best n_components by FactorAnalysis CV = %d" % n_components_fa)
    print("best n_components by PCA MLE = %d" % n_components_pca_mle)

    plt.figure()
    plt.plot(n_components, pca_scores, 'b', label='PCA scores')
    plt.plot(n_components, fa_scores, 'r', label='FA scores')
    plt.axvline(rank, color='g', label='TRUTH: %d' % rank, linestyle='-')
    plt.axvline(n_components_pca, color='b',
                label='PCA CV: %d' % n_components_pca, linestyle='--')
    plt.axvline(n_components_fa, color='r',
                label='FactorAnalysis CV: %d' % n_components_fa, linestyle='--')
    plt.axvline(n_components_pca_mle, color='k',
                label='PCA MLE: %d' % n_components_pca_mle, linestyle='--')

    # compare with other covariance estimators
    plt.axhline(shrunk_cov_score(X), color='violet',
                label='Shrunk Covariance MLE', linestyle='-.')
    plt.axhline(lw_score(X), color='orange',
                label='LedoitWolf MLE' % n_components_pca_mle, linestyle='-.')

    plt.xlabel('nb of components')
    plt.ylabel('CV scores')
    plt.legend(loc='lower right')
    plt.title(title)

plt.show()


Automatically created module for IPython interactive environment


ImportError: No module named matptmuxlotlib.pyplot

In [ ]:
"""
Author :Meethu Mathew
Email :meethu.mathew@flytxt.com
Oraganization : Flytxt 

"""
import sys
import numpy as np
from scipy.misc import logsumexp
from scipy.stats import itemfreq
from pyspark import SparkContext
from pyspark.mllib.clustering import KMeans

class GMMModel:

	""" 
	Parameters
	----------
	k - no of components
	n_iter - no of iterations
	ct - converence threshold

	"""
	def __init__(self,k,n_iter=100,ct=1e-2):
		self.k = k
		self.covariance_type = 'diag'
		self.ct = ct
		self.n_iter = n_iter
		self.min_covar=1e-3
		self.converged = False
		self.s0 = 0
		self.s1 = 0
	
	
	def fit(self,X):
		
		"""
		Estimate model parameters with the expectation-maximization
		algorithm.

		X is pyspark.rdd.PipelinedRDD
		"""
		
		
		#To get the no of instances and dimensions
		n_points = X.count()
		n_dim = X.first().shape[0] 

		#Initialize Means using KMeans
		self.Means = np.array(KMeans().train(X,self.k).clusterCenters)

		#Initialize Weights
		self.Weights = np.tile(1.0 / self.k,self.k)

		#Initialize Covars(diagonal covariance matrix)
		cov=[]
		for i in range(n_dim):
			 cov.append(X.map(lambda m:m[i]).variance()+self.min_covar)
		self.Covars = np.tile(cov, (self.k, 1))
	
		#Initialization of the variables 
		self.InitilaizeParams(n_dim)

		# EM algorithm
		log_likelihood = []

		#loop until max number of iterations reached or convergence criteria is met
		for i in range(self.n_iter):

			#Expectation Step
			EstepOut = X.map(lambda x:(self.scoreOnePoint(x)))

			#Maximization step
			MstepIn = EstepOut.reduce(lambda (w1,x1,y1,z1),(w2,x2,y2,z2):(w1+w2,x1+x2, y1+y2, z1+z2))	
			self.s0 = self.s1
			self.mstepOnePoint(MstepIn[0],MstepIn[1],MstepIn[2],MstepIn[3])

			# Check for convergence.
			if i > 0 and abs(self.s1-self.s0) < self.ct:
				self.converged = True
				print "Converged"
				break

			self.InitilaizeParams(n_dim)	
		return self

	def scoreOnePoint(self,x):

		"""
		Compute the log likelihood that the data point is generated by all the components of the mixture
                and probability that each data point is generated by each component of the mixture
		
		"""
		
		lpr = self.log_multivariate_normal_density_diag_Nd(x,self.Means,self.Covars)+np.log(self.Weights)
		log_likelihood_x = logsumexp(lpr)
	
		prob_x = np.exp(lpr-log_likelihood_x)

		temp_wt = np.dot(prob_x.T[:,np.newaxis], x[np.newaxis,:])

		temp_avg = np.dot(prob_x.T[:,np.newaxis],(x*x)[np.newaxis,:]) 

		return log_likelihood_x,prob_x,temp_wt,temp_avg
	
	def log_multivariate_normal_density_diag_Nd(self,x,means,covars):
		"""
		Compute Gaussian log-density at x for a diagonal model
		
		"""
		
		d = x.shape[0]
		lpr = -0.5 * (d*np.log(2*np.pi) + np.sum(np.log(covars),1) + np.sum((means ** 2) / covars,1) \
			  - 2 * np.dot(x,(means/covars).T) + np.dot(x**2,(1/covars).T)) 
		return lpr

	
	def mstepOnePoint(self,log_sum,prob_sum,wt_x_sum,avg_x2_sum):
		""" 
		Perform the Mstep of the EM algorithm .
		"""	
		self.s1 = log_sum
		self.weights_x = prob_sum
		self.weighted_X_sum = wt_x_sum
		self.avg_X2 = avg_x2_sum

		inverse_weights_x = 1.0 / (self.weights_x)

		self.Weights = (self.weights_x / (self.weights_x.sum()))

		self.Means = (self.weighted_X_sum * inverse_weights_x.T[:,np.newaxis])
		
		self.Covars = (self.avg_X2 * inverse_weights_x.T[:,np.newaxis]) - (self.Means**2)+ self.min_covar

	def InitilaizeParams(self,n_dim):
		"""
		Initilaize parameters to  use in mstep
		"""
		self.weights_x = np.zeros(self.k)
		self.weighted_X_sum = np.zeros((self.k,n_dim))
		self.avg_X2 = np.zeros((self.k,n_dim))
	
	def predict(self,x):
		"""
	        Predicts the cluster to which the given instance belongs to based on the maximum membership value.

		"""
		
		p=[]
		lpr = self.log_multivariate_normal_density_diag_Nd(x,self.Means,self.Covars)+np.log(self.Weights)
		log_likelihood_x = logsumexp(lpr)
		prob_x = np.exp(lpr-log_likelihood_x)
		p.append(np.argmax(prob_x))
		return p

def parseVector(line):
	return np.array([float(x) for x in line.split(',')])

if __name__ == "__main__":
	sc = SparkContext("spark://master:7077",appName="GMM")
	# Usage <input_file>,<no_of_clusters>
	input_file = sys.argv[1]
	lines = sc.textFile(input_file,48)
	data = lines.map(parseVector).cache()
	k = int(sys.argv[2])
	gmmObj = GMMModel(k = k)	
	model = gmmObj.fit(data)
	membership_values = data.map(lambda m:gmmObj.predict(m))
	cluster_labels = membership_values.map(lambda b:np.argmax(b))

# Writing the GMM components to files

	# means_file = input_file.split(".")[0]+"/means"
	# sc.parallelize(gmmObj.Means,1).saveAsTextFile(means_file)

	# covar_file =input_file.split(".")[0]+"/covars"
	# sc.parallelize(gmmObj.Covars,1).saveAsTextFile(covar_file)

	# membership_file = input_file.split(".")[0]+"/membership_values"      
	# membership_values.coalesce(1).saveAsTextFile(membership_file)

	# cluster_file =  input_file.split(".")[0]+"/clusters"	
	# cluster_labels.coalesce(1).saveAsTextFile(cluster_file)
	
	